<a href="https://colab.research.google.com/github/jonisabdulkarim/CE888/blob/main/Lab8/TF_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, datasets
from zipfile import ZipFile

In [2]:
!wget https://github.com/jonisabdulkarim/CE888/raw/main/Lab8/sorted.zip

--2021-03-19 14:34:56--  https://github.com/jonisabdulkarim/CE888/raw/main/Lab8/sorted.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jonisabdulkarim/CE888/main/Lab8/sorted.zip [following]
--2021-03-19 14:34:57--  https://raw.githubusercontent.com/jonisabdulkarim/CE888/main/Lab8/sorted.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20163420 (19M) [application/zip]
Saving to: ‘sorted.zip’

sorted.zip          100%[===================>]  19.23M  47.4MB/s    in 0.4s    

2021-03-19 14:34:57 (47.4 MB/s) - ‘sorted.zip’ saved [20163420/20163420]



In [3]:
file_name = "/content/sorted.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [4]:
# constant values
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
BATCH_SIZE = 8
SEED = 123
VALIDATION_SPLIT = 0.1
LABELS = 'inferred'
LABEL_MODE = 'int'

In [5]:
# import images for training
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='/content/sorted/train',
    color_mode='rgb',
    labels=LABELS,
    label_mode=LABEL_MODE,
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=SEED,
    batch_size=BATCH_SIZE
)
class_names = train_ds.class_names

# import images for validation
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='/content/sorted/train',
    color_mode='rgb',
    labels=LABELS,
    label_mode=LABEL_MODE,
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=SEED,
    batch_size=BATCH_SIZE
)

# import images for hold-out test
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='/content/sorted/test',
    color_mode='rgb',
    labels=LABELS,
    label_mode=LABEL_MODE,
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE
)

Found 716 files belonging to 4 classes.
Using 645 files for training.
Found 716 files belonging to 4 classes.
Using 71 files for validation.
Found 92 files belonging to 4 classes.


In [6]:
print(class_names)

['Humans', 'cats', 'dogs', 'horses']


In [44]:
## VGG 16

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model

img_input = Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))

In [45]:
data_augmentation = layers.experimental.preprocessing.Rescaling(1.0 / 255)

In [46]:
first = data_augmentation(img_input)

In [47]:
model = VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=first,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling_6 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584

In [48]:
last_layer = model.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(64, activation='relu', name='fc2')(x)
out = Dense(4, activation='softmax', name='output')(x)  ## 4 classes
model = Model(img_input, out)

for layer in model.layers[:-3]:
	layer.trainable = False

model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling_6 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     1475

In [49]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [50]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
my_callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='vgg16_model.h5', save_best_only=True),
]

In [51]:
history = model.fit(train_ds, epochs=20, 
                              validation_data=val_ds,
                              callbacks=my_callbacks)

Epoch 1/20
81/81 [==============================] - 347s 4s/step - loss: 1.3071 - acc: 0.5755 - val_loss: 0.2175 - val_acc: 0.9577
Epoch 2/20
81/81 [==============================] - 348s 4s/step - loss: 0.1090 - acc: 0.9585 - val_loss: 0.2669 - val_acc: 0.8873
Epoch 3/20
81/81 [==============================] - 349s 4s/step - loss: 0.0419 - acc: 0.9924 - val_loss: 0.1793 - val_acc: 0.9577
Epoch 4/20
81/81 [==============================] - 346s 4s/step - loss: 0.0136 - acc: 1.0000 - val_loss: 0.1642 - val_acc: 0.9577
Epoch 5/20
81/81 [==============================] - 345s 4s/step - loss: 0.0031 - acc: 1.0000 - val_loss: 0.1740 - val_acc: 0.9577
Epoch 6/20
67/81 [=======================>......] - ETA: 54s - loss: 0.0019 - acc: 1.0000

KeyboardInterrupt: ignored